# Часть первая: крестики-нолики при помощи Q-обучения

Почти везде графики обучения и результаты будут приведены в Tensorboard. Их можно открыть по ссылке и посмотреть интерактивные графики.

При разгрузке были какие-то технические ошибки, поэтому некоторые графики обрываются раньше других. 

## Противники

Для окружения размера 3x3 был построено минимакс дерево с альфа-бета отсечением. Для окружения большего размера была попытка построить MCTS дерево и использовать его в качестве противника при обучении всех алгоритмов. 

Для окружения размера 4x4 результаты по игре оказались не сильно лучше чем при случайном противнике, поэтому было принято решение использовать случайную стратегию, чтобы быстрее считать и меньше места занимать в оперативной памяти. Для окружения 4x4 при полном расширении требовалось слишком много оперативной памяти. Была попытка урезать глубину дерева и не делать полное расширение, чтобы сэкономить, но результаты оказались не сильно лучше при игре, чем случайные стратегии. Для полного построения MCTS дерева был ещё проверен вариант с построением графа вместо дерева, чтобы переиспользовать одинаковые позиции на поле, но библиотека для работы с графами оказалась намного медленне, чем с деревьями.

Минимакс дерево было немного рандомезировано, чтобы не периодически совершать случайные действия и не всегда следовать оптимальному алгоритму.

Для проверки правильности построения и работы минимакс дерева проверялся фак того что если оба игрока следуют минимакс стратегии, то всегда получается ничья.

In [131]:
import pathlib
import json
import pandas as pd

In [132]:
%load_ext autoreload
%autoreload 2

from notebook_utils import show_tables, game_stat_to_table

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Результаты игры рандомизированного минимакс дерева против случайного противника.

In [133]:
stat_dir = pathlib.Path("./stat/cross_minmax_vs_random")

### Минимакс за крестиков против случайно стратегии

In [134]:
show_tables(stat_dir)

Вероятность совершить случайное действие,Доля побед крестиков,Доля побед ноликов,Доля ничьих
0,0.979,0,0.021
0.1,0.923,0.0365,0.0405
0.2,0.8505,0.0855,0.064
0.5,0.753,0.164,0.083
0.7,0.6775,0.205,0.1175
0.9,0.6245,0.268,0.1075


### Минимакс за ноликов против случайно стратегии

In [135]:
stat_dir = pathlib.Path("./stat/cross_random_vs_minmax")
show_tables(stat_dir)

Вероятность совершить случайное действие,Доля побед крестиков,Доля побед ноликов,Доля ничьих
0,0,0.7965,0.2035
0.1,0.175,0.6695,0.1555
0.2,0.2705,0.5965,0.133
0.5,0.4405,0.439,0.1205
0.7,0.506,0.3555,0.1385
0.9,0.5585,0.3235,0.118


В итоге была выбрана вероятность равная 0,09, чтобы дать возможность иногда побеждать противнику. 

Далее структура отчёты будет одинакова:
* ссылка на результаты обучения при различных гиперпараметрах (представлены результаты игр);
* ссылка на графики с общей динамикой при различных параметрах;
* ссылка на динамику обучения с наилучшими параметрами.

Описание параметров:
* `valid_cross_win` - доля побед за крестиков после обучения;
* `valid_circle_win` - доля побед за ноликов после обучения;
* `valid_draw` - доля ничьих после обучения.

Значение на графиках с префиксом `Train` это значения аналогичных метрик только во время обучения для отслеживания общей динамики.


* `alpha` - в случае нейронных сетей learning_rate метода оптимизации;
* `train_mean_reward`- это усреднённая награда во время обучения.

**В Q-обучении везде использовалась epsilon-greedy стратегия с epsilon = 0,02. Это могло повлиять иногда на результаты игр** 

## Табличное Q-обучение

### Окружение 3x3

#### Крестики используют Q-обучение - нолики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/ha5nkkkqQKWvyGcWqhEcMg/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/ha5nkkkqQKWvyGcWqhEcMg/#scalars&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/ha5nkkkqQKWvyGcWqhEcMg/#scalars&_smoothingWeight=0.999&regexInput=0_)

#### Крестики рандомизированный минимакс - нолики Q-обучение  

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/75297X18RxCvKEoCDVvqgQ/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/75297X18RxCvKEoCDVvqgQ/#scalars&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/75297X18RxCvKEoCDVvqgQ/#scalars&_smoothingWeight=0.999&regexInput=2_)

### Окружение 4x4

#### Крестики используют Q-обучение - нолики случайная стратегия

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/Wnr9dWJJRayogGIeupRJaQ/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/Wnr9dWJJRayogGIeupRJaQ/#scalars&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/Wnr9dWJJRayogGIeupRJaQ/#scalars&_smoothingWeight=0.999&regexInput=1_)

#### Крестики используют случайную стратегию - нолики Q-обучение

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/04d6vY0JQNW83lFBk9hQ5w/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/04d6vY0JQNW83lFBk9hQ5w/#scalars&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/04d6vY0JQNW83lFBk9hQ5w/#scalars&_smoothingWeight=0.999&regexInput=6_)

Тут появились проблемы, если смотреть на графики крупным планом, то некоторая сходимость наблюдается, но очень медленная. Возможно, нужно было намного больше эпизодов, чтобы покрыть большое число состояний и сделать какую-то оценку для дискретных значений. 

За ноликов оказалось совсем сложно обучать.

# Часть вторая DQN и Dueling DQN 

Для DQN и Dueling DQN добавляется дополнительная метрика это `L1_loss`. Значение функции потерь, которая использовалась для оптимизации.

## DQN

### Окружение 3x3

#### Крестики используют DQN - нолики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/UmT39gByTQmSm5VdFVdE6Q/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/UmT39gByTQmSm5VdFVdE6Q/#scalars&_smoothingWeight=0.999&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg4NjE4Mi41NjQ3NDQ3Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3ODg2MzcxLjM0MzMzODMiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzg4NjIwNi4yOTMzNDgzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg4NjM3OC4wMTg4OTAxIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzg4NjA2My40OTQ4NjciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc4ODYwMzQuNjYxNzk5NCI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc4ODYxNzkuODExMTY1OCI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc4ODYwMjQuOTQwMzE5NSI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvMTYzNzg4NjM3MS4wMTc5MDIiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/UmT39gByTQmSm5VdFVdE6Q/#scalars&regexInput=2_&_smoothingWeight=0.999)

#### Нолики используют DQN - крестики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/W91IJbPRRXKW0FL36rSbNg/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/W91IJbPRRXKW0FL36rSbNg/#scalars&_smoothingWeight=0.999&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg5NDQwNy42MDcyNDIzIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3ODk0NDMzLjcxNjk0MDYiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzg5NDQwNC4xMTUzOTQiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy8xNjM3ODk0MzY5LjcxMTQ3ODUiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy8xNjM3ODk0MjYyLjA4Mzg1MTgiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc4OTQyMTcuMDMyMTkzIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg5NDMwNy42NTQ1NTQ0Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvMTYzNzg5NDQ4Mi43NTgzMTg3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3ODk0MTc3LjQ3NDA5MzIiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/W91IJbPRRXKW0FL36rSbNg/#scalars&_smoothingWeight=0.999&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg5NDQwNy42MDcyNDIzIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3ODk0NDMzLjcxNjk0MDYiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzg5NDQwNC4xMTUzOTQiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy8xNjM3ODk0MzY5LjcxMTQ3ODUiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy8xNjM3ODk0MjYyLjA4Mzg1MTgiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc4OTQyMTcuMDMyMTkzIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvMTYzNzg5NDMwNy42NTQ1NTQ0Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvMTYzNzg5NDQ4Mi43NTgzMTg3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3ODk0MTc3LjQ3NDA5MzIiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D&regexInput=%5E2_)

На ноликов с DQN оказалось намного сложнее. Результат по мере улучшается, но похоже на то что нужно намного больше итераций. 

### Окружение 4x4

#### Крестики используют DQN - нолики случайную стратегию

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/0gb5KubmT96RLgRbeA6nxQ/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/0gb5KubmT96RLgRbeA6nxQ/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcMTYzNzkxNTY4My4zNDQ3ODM1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTE1NzA2Ljg2MDk0NSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFwxNjM3OTE1Nzk2LjAyOTUyNDgiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTE1NjQxLjk5MjAwNSI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXDE2Mzc5MTU4MDYuNTIwODYzOCI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjVcXGxvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzkxNTY1OS4wMTQxMzYiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTE1NzY2LjM1OTAwNTUiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTE1NzIwLjA0NjQ5MjYiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXDE2Mzc5MTU3MzcuMjUxMTgyMyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/0gb5KubmT96RLgRbeA6nxQ/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcMTYzNzkxNTY4My4zNDQ3ODM1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTE1NzA2Ljg2MDk0NSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFwxNjM3OTE1Nzk2LjAyOTUyNDgiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTE1NjQxLjk5MjAwNSI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXDE2Mzc5MTU4MDYuNTIwODYzOCI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjVcXGxvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzkxNTY1OS4wMTQxMzYiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTE1NzY2LjM1OTAwNTUiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTE1NzIwLjA0NjQ5MjYiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXDE2Mzc5MTU3MzcuMjUxMTgyMyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999&regexInput=%5E5_)

#### Нолики используют DQN - крестики случайную стратегию

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/n1gjL4APR5qiMml5o8XHBw/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/n1gjL4APR5qiMml5o8XHBw/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzkxOTE5OC42OTc1NCI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvMTYzNzkxOTIzOS45NzQyMDY3Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5MTkyMDQuODgwMDUyOCI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzLzE2Mzc5MTkyMDguMTk0ODQzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzkxOTIwMS4wMzI1NzQiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5MTkyMjguOTcxODkzMyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc5MTkwNjMuNjkwMDA2NyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc5MTkwMzQuNDYzMjExIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3OTE5MjQyLjYwNjA2NSI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/n1gjL4APR5qiMml5o8XHBw/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzkxOTE5OC42OTc1NCI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvMTYzNzkxOTIzOS45NzQyMDY3Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5MTkyMDQuODgwMDUyOCI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzLzE2Mzc5MTkyMDguMTk0ODQzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzkxOTIwMS4wMzI1NzQiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5MTkyMjguOTcxODkzMyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc5MTkwNjMuNjkwMDA2NyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc5MTkwMzQuNDYzMjExIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3OTE5MjQyLjYwNjA2NSI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlfQ%3D%3D&_smoothingWeight=0.999&regexInput=%5E0_)

## Dueling DQN 

### Окружение 3x3

#### Крестики используют Dueling DQN - нолики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/XYnqvc9VTmuya3Rh7Qvk4Q/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/XYnqvc9VTmuya3Rh7Qvk4Q/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcMTYzNzk2MjU1MC44NTA2MTAzIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyNTg5LjQyMTE3MDciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzk2MjY3MS43NzUxMTg4Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcMTYzNzk2MjgwOC4xMTQxMzIiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyMzU5Ljg5MDAzMyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjVcXGxvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzk2MjQxNC41MTczNTIiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTYyNzc4LjcyNjMxNzIiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyNzQwLjUyMjQ0MDQiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXDE2Mzc5NjI0MTguMzIxNjM4Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2RyYXciOnRydWV9&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/XYnqvc9VTmuya3Rh7Qvk4Q/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcMTYzNzk2MjU1MC44NTA2MTAzIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjFcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9ncyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyNTg5LjQyMTE3MDciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzk2MjY3MS43NzUxMTg4Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcMTYzNzk2MjgwOC4xMTQxMzIiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyMzU5Ljg5MDAzMyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjVcXGxvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcMTYzNzk2MjQxNC41MTczNTIiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1XFxsb2dzXFxUcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjFcXGxvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFwxNjM3OTYyNzc4LjcyNjMxNzIiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xXFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjVcXGxvZ3MiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFwxNjM3OTYyNzQwLjUyMjQ0MDQiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41XFxsb2dzXFxUcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNVxcbG9nc1xcVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1XFxsb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXDE2Mzc5NjI0MTguMzIxNjM4Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNVxcbG9nc1xcVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjVcXGxvZ3NcXFRyYWluX2RyYXciOnRydWV9&_smoothingWeight=0.999&regexInput=%5E1_)

#### Нолики используют Dueling DQN - крестики случайную стратегию

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/fv0yyTq4TRawmoPvAEJzMA/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/fv0yyTq4TRawmoPvAEJzMA/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk3ODgwMy42NDM0MzU1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3OTc4NzM2Ljg0Mzk2NjIiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzk3ODQ2NS4yMDI3MzczIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk3ODc1NC41NjgxMzM0Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzk3ODM4Ni45MDU1NzciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5Nzg1NzkuMjE5MjgyOSI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc5Nzg2NDYuODI4OTQ2OCI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc5Nzg1MDcuMTY0NjA2Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3OTc4NDY4Ljk0NTU4NzYiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/fv0yyTq4TRawmoPvAEJzMA/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk3ODgwMy42NDM0MzU1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM3OTc4NzM2Ljg0Mzk2NjIiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzk3ODQ2NS4yMDI3MzczIjp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk3ODc1NC41NjgxMzM0Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjNfYWxwaGE9MWUtMDVfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvMTYzNzk3ODM4Ni45MDU1NzciOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjRfYWxwaGE9MWUtMDVfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5Nzg1NzkuMjE5MjgyOSI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzLzE2Mzc5Nzg2NDYuODI4OTQ2OCI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI2X2FscGhhPTFlLTA2X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzLzE2Mzc5Nzg1MDcuMTY0NjA2Ijp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy8xNjM3OTc4NDY4Ljk0NTU4NzYiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZX0%3D&_smoothingWeight=0.999&regexInput=%5E0_)

Тут рказалось сложным обучить модель с Dueling DQN за ноликов с точки зреняи побед. По большой части наблюдается росто числа ничьих. Победы растут, но не бстро. Тут либо надо больше времени на обучение, либо лругие гиперпараметры выбрать.

### Окружение 4x4

#### Крестики используют Dueling DQN - нолики рандомизированный минимакс

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/EbeKKjklSq6aphmxNQx2iA/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/EbeKKjklSq6aphmxNQx2iA/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk4MDIwOC45NzkzNjEiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzLzE2Mzc5ODAzNjIuMjk2OTg1Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5ODAzMDIuMzE4MTMwMyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzLzE2Mzc5ODAzNzcuNTIxMzg0NSI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzLzE2Mzc5ODA0NjIuMzgwNDMxNyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzk4MDMxMC4zNDAwNzkzIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk4MDQzMC4yNDI3NjMiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy8xNjM3OTgwNDQwLjA0NjAyOSI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvMTYzNzk4MDM3MS42MzY4OTY4Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWV9&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/EbeKKjklSq6aphmxNQx2iA/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk4MDIwOC45NzkzNjEiOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MiOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzLzE2Mzc5ODAzNjIuMjk2OTg1Ijp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzLzE2Mzc5ODAzMDIuMzE4MTMwMyI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzLzE2Mzc5ODAzNzcuNTIxMzg0NSI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzLzE2Mzc5ODA0NjIuMzgwNDMxNyI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvMTYzNzk4MDMxMC4zNDAwNzkzIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvMTYzNzk4MDQzMC4yNDI3NjMiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy8xNjM3OTgwNDQwLjA0NjAyOSI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI3X2FscGhhPTFlLTA2X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MvMTYzNzk4MDM3MS42MzY4OTY4Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWV9&_smoothingWeight=0.999&regexInput=%5E0_)

#### Нолики используют Dueling DQN - крестики случайную стратегию

[Результаты при различных гиперпараметрах](https://tensorboard.dev/experiment/oT1ArOTETlGpBbfmbI43Ug/#hparams)

[Общий тренд обучения при различных гиперпараметрах](https://tensorboard.dev/experiment/oT1ArOTETlGpBbfmbI43Ug/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzODAwNTAyNi40NTk0NzU1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM4MDA1MDkyLjg4MDI2NSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy8xNjM4MDA1MDkxLjk5OTc3MSI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzLzE2MzgwMDQ5OTUuMDY3NzE5NyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzLzE2MzgwMDQ4MjcuOTYyNzU4NSI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvMTYzODAwNDkyOS44MzEzNTEiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy8xNjM4MDA1MDY2LjM5NTgyOTIiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy8xNjM4MDA0OTg5LjAyNjE2NzYiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzLzE2MzgwMDUwNTYuNjk1NDc1Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWV9&_smoothingWeight=0.999)

[Динамика обучения с наилучшими параметрами](https://tensorboard.dev/experiment/oT1ArOTETlGpBbfmbI43Ug/#scalars&runSelectionState=eyIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiMF9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4xL2xvZ3MvMTYzODAwNTAyNi40NTk0NzU1Ijp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIwX2FscGhhPTAuMDAwMV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjBfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMS9sb2dzL1RyYWluX2RyYXciOnRydWUsIjFfYWxwaGE9MC4wMDAxX2dhbW1hPTAuNS9sb2dzIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy8xNjM4MDA1MDkyLjg4MDI2NSI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiMV9hbHBoYT0wLjAwMDFfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY3Jvc3Nfd2luIjp0cnVlLCIxX2FscGhhPTAuMDAwMV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIyX2FscGhhPTAuMDAwMV9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy8xNjM4MDA1MDkxLjk5OTc3MSI6dHJ1ZSwiMl9hbHBoYT0wLjAwMDFfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjJfYWxwaGE9MC4wMDAxX2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzLzE2MzgwMDQ5OTUuMDY3NzE5NyI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCIzX2FscGhhPTFlLTA1X2dhbW1hPTAuMS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiM19hbHBoYT0xZS0wNV9nYW1tYT0wLjEvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzLzE2MzgwMDQ4MjcuOTYyNzU4NSI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI0X2FscGhhPTFlLTA1X2dhbW1hPTAuNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNF9hbHBoYT0xZS0wNV9nYW1tYT0wLjUvbG9ncy9UcmFpbl9kcmF3Ijp0cnVlLCI1X2FscGhhPTFlLTA1X2dhbW1hPTAuMjUvbG9ncyI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvMTYzODAwNDkyOS44MzEzNTEiOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2NpcmNsZV93aW4iOnRydWUsIjVfYWxwaGE9MWUtMDVfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2Nyb3NzX3dpbiI6dHJ1ZSwiNV9hbHBoYT0xZS0wNV9nYW1tYT0wLjI1L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncyI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy8xNjM4MDA1MDY2LjM5NTgyOTIiOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiNl9hbHBoYT0xZS0wNl9nYW1tYT0wLjEvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjZfYWxwaGE9MWUtMDZfZ2FtbWE9MC4xL2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncyI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy8xNjM4MDA0OTg5LjAyNjE2NzYiOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fY2lyY2xlX3dpbiI6dHJ1ZSwiN19hbHBoYT0xZS0wNl9nYW1tYT0wLjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjdfYWxwaGE9MWUtMDZfZ2FtbWE9MC41L2xvZ3MvVHJhaW5fZHJhdyI6dHJ1ZSwiOF9hbHBoYT0xZS0wNl9nYW1tYT0wLjI1L2xvZ3MiOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzLzE2MzgwMDUwNTYuNjk1NDc1Ijp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jaXJjbGVfd2luIjp0cnVlLCI4X2FscGhhPTFlLTA2X2dhbW1hPTAuMjUvbG9ncy9UcmFpbl9jcm9zc193aW4iOnRydWUsIjhfYWxwaGE9MWUtMDZfZ2FtbWE9MC4yNS9sb2dzL1RyYWluX2RyYXciOnRydWV9&_smoothingWeight=0.999&regexInput=%5E0_)

# Часть третья: расширим и углубим поиск

Далее приведены результаты игры разных стратегий между собой в том числе и с MCTS деревом при различных комбинациях rollout. 

MCTS дерево реализована в режиме online. Можно обновлять статистики по мере игры или заранее сыграть несколько игр, а потом использовать дерево обновляя статистики в реальном времени. В качестве критерия для выбора узлов на этапе selection используется UCB1. 

In [136]:
stat_dir = pathlib.Path("./stat/game_simul/3")

## Для окружения 3x3

In [137]:
game_stat_3x3 = game_stat_to_table(stat_dir)

In [138]:
cross_win_col_name = "Доля побед крестиков"
top_n = 10

Максимальная доля побед за крестиков

In [139]:
game_stat_3x3.nlargest(n=top_n, columns=cross_win_col_name)

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
40,q-table_vs_dqn,0.9880,0.0010,0.0110
17,mcts-rollout-dueling-dqn_vs_dueling-dqn,0.9870,0.0110,0.0020
44,q-table_vs_mcts-rollout-random,0.9840,0.0035,0.0125
43,q-table_vs_mcts-rollout-q-table,0.9830,0.0020,0.0150
42,q-table_vs_mcts-rollout-dueling-dqn,0.9825,0.0030,0.0145
41,q-table_vs_dueling-dqn,0.9820,0.0010,0.0170
47,q-table_vs_random,0.9815,0.0035,0.0150
1,dqn_vs_dueling-dqn,0.9760,0.0060,0.0180
33,mcts-rollout-random_vs_dueling-dqn,0.9720,0.0250,0.0030
50,rand-minmax_vs_mcts-rollout-dueling-dqn,0.9325,0.0305,0.0370


In [140]:
circle_win_col_name =  "Доля побед ноликов"

Максимальная доля побед за ноликов

In [141]:
game_stat_3x3.nlargest(n=top_n, columns=circle_win_col_name)

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
29,mcts-rollout-q-table_vs_q-table,0.1810,0.6745,0.1445
62,random_vs_rand-minmax,0.1720,0.6560,0.1720
61,random_vs_q-table,0.1880,0.5945,0.2175
30,mcts-rollout-q-table_vs_rand-minmax,0.2685,0.4790,0.2525
56,random_vs_dqn,0.1370,0.4240,0.4390
38,mcts-rollout-random_vs_rand-minmax,0.3230,0.3970,0.2800
22,mcts-rollout-dueling-dqn_vs_rand-minmax,0.3370,0.3900,0.2730
24,mcts-rollout-q-table_vs_dqn,0.1985,0.3470,0.4545
37,mcts-rollout-random_vs_q-table,0.5485,0.3345,0.1170
60,random_vs_mcts-rollout-random,0.5690,0.2995,0.1315


In [142]:
draw_col_name = "Доля ничьих"

Максимальная доля ничьих

In [143]:
game_stat_3x3.nlargest(n=top_n, columns=draw_col_name)

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
0,dqn_vs_dqn,0.0060,0.0080,0.9860
13,dueling-dqn_vs_q-table,0.0115,0.0085,0.9800
8,dueling-dqn_vs_dqn,0.0210,0.0000,0.9790
45,q-table_vs_q-table,0.0250,0.0070,0.9680
5,dqn_vs_q-table,0.0175,0.0150,0.9675
9,dueling-dqn_vs_dueling-dqn,0.0365,0.0010,0.9625
14,dueling-dqn_vs_rand-minmax,0.3230,0.0130,0.6640
53,rand-minmax_vs_q-table,0.0625,0.2810,0.6565
16,mcts-rollout-dueling-dqn_vs_dqn,0.2420,0.1020,0.6560
49,rand-minmax_vs_dueling-dqn,0.2600,0.1110,0.6290


In [144]:
game_stat_3x3[game_stat_3x3["Стратегии игроков"].str.contains("mcts")]

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
2,dqn_vs_mcts-rollout-dueling-dqn,0.7075,0.0245,0.2680
3,dqn_vs_mcts-rollout-q-table,0.7020,0.0170,0.2810
4,dqn_vs_mcts-rollout-random,0.7060,0.0230,0.2710
10,dueling-dqn_vs_mcts-rollout-dueling-dqn,0.7575,0.0125,0.2300
11,dueling-dqn_vs_mcts-rollout-q-table,0.7670,0.0075,0.2255
12,dueling-dqn_vs_mcts-rollout-random,0.7665,0.0120,0.2215
16,mcts-rollout-dueling-dqn_vs_dqn,0.2420,0.1020,0.6560
17,mcts-rollout-dueling-dqn_vs_dueling-dqn,0.9870,0.0110,0.0020
18,mcts-rollout-dueling-dqn_vs_mcts-rollout-dueli...,0.9055,0.0330,0.0615
19,mcts-rollout-dueling-dqn_vs_mcts-rollout-q-table,0.8830,0.0480,0.0690


In [145]:
HTML(game_stat_3x3.to_html())

,Стратегии игроков,Доля побед крестиков,Доля побед ноликов,Доля ничьих
0,dqn_vs_dqn,0.0060,0.0080,0.9860
1,dqn_vs_dueling-dqn,0.9760,0.0060,0.0180
2,dqn_vs_mcts-rollout-dueling-dqn,0.7075,0.0245,0.2680
3,dqn_vs_mcts-rollout-q-table,0.7020,0.0170,0.2810
4,dqn_vs_mcts-rollout-random,0.7060,0.0230,0.2710
5,dqn_vs_q-table,0.0175,0.0150,0.9675
6,dqn_vs_rand-minmax,0.3260,0.0675,0.6065
7,dqn_vs_random,0.6915,0.0240,0.2845
8,dueling-dqn_vs_dqn,0.0210,0.0000,0.9790
9,dueling-dqn_vs_dueling-dqn,0.0365,0.0010,0.9625
